# Grab land cover for specified extent
Say you want to grab land cover data for a specific location, but you don't want to download the entire NLCD dataset to do that. The ArcGIS Python API can help!

Here I present an example of doing just that. The steps involve:
* Prepping for the analysis: importing modules and authenticating our arcgis session
* Locating the land cover data **item** and creating a **imagery layer** from it
* 

*Resource: https://developers.arcgis.com/python/guide/raster-analysis-advanced-concepts/*

### Prepping for analysis: importing and authenticating
* Import the the arcgis `GIS` module. We're going to do some geocoding so we need to create the arcgis `geocoding` module as well. Lastly, enable the Jupyter `display` object

In [ ]:
#Import the GIS object.  and display modules
from arcgis import GIS
#from arcgis.geocoding import geocode
from IPython.display import display, Image

* Authenticate our GIS object using our ArcGIS Pro account

In [ ]:
#Create the GIS object, authenticating with your ArcGIS Pro account
gis = GIS('pro')

### Searching for the content and linking to it
What we want is 2011 NLCD data, provided as an *image service* (i.e. as a raster). You could search for the data from within ArcGIS Pro or via the [ArcGIS Online](http://www.arcgis.com) website, but we'll do it right here. 

Like any web search, it's a bit of an art knowing how best to locate the resource you want. At play are what general search keywords to include, and specific categories like `owner` or `item-type` to invoke. We do, however, want to search outside the Duke Community, we we want to include `outside_org=True`. 

I've decided to use `NLCD 2011` as a general search term, filter results for only those that `esri` provides, and limit results to image services: 

In [ ]:
#Search for land cover and print the number of results
lc_results = gis.content.search("NLCD 2011, owner:esri",item_type='image',outside_org=True)
len(lc_results)

► This gives us 10 results, enough to show a list...

In [ ]:
#Show a complete list of results
lc_results

* The *second* item is the one we want. Let's store that as a variable named `lc_item`. 

In [ ]:
#Get the second result and view what kind of object it is. 
lc_item = lc_results[1]
type(lc_item)

In [ ]:
#Now let's view the item...
lc_item

* Let's examine a few **properties** of this item. Because arcgis Item objects are dynamic (they can be vector or raster or...) the list of what properties can change. But we can get a list of properies via the `item.keys()` function:

In [ ]:
#List the property keys associated with the item we fetched
lc_item.keys()

* Let's examine the services' web address or its *URL*. Open this URL in your web browser. (Note, you'll have to authenticate as this layer is only available to ESRI license holders!) 

In [ ]:
#Get the URL
print(lc_item['url']) #we could also use lc_item.url

* Down the road, we may need the image's **spatial reference**, so let's store that as a variable.

In [ ]:
#Extract the image service's spatial reference to a variable
lc_sr = lc_item.spatialReference
lc_sr

### From *arcgis* [`item`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#item) to *arcgis* [`layer`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#layer) (specifically an [`imagery layer`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.raster.toc.html#imagerylayer))

* Now we need to extract the data **layer** from the data **item**. The `layers` property returns a list of layers associated with this image service, of which there is only one. So we extract that one to a new variable called `lc_lyr`...<br>*Calling this variable displays it to our notebook!*

In [ ]:
#Extract the one (and only) layer in the item to a new variable and display it
lc_lyr = lc_item.layers[0]
type(lc_lyr)

In [ ]:
lc_lyr

In [ ]:
#What are the pixel sizes in the X and Y direction? 
xPixelSize = lc_lyr.properties.pixelSizeX
yPixelSize = lc_lyr.properties.pixelSizeY
print(xPixelSize,yPixelSize)

## Subsetting our image
* Define an extent and convert it into an ArcGIS "geometry" object
 * Will need to import the arcgis geometry module...

In [ ]:
#create a dictionary of extent values, in decimal degrees
myExtent = {'xmin': -79.15,
            'ymin': 35.95,
            'xmax': -78.95,
            'ymax': 36.15}

In [ ]:
#Import the ArcGIS geometry module
from arcgis import geometry

In [ ]:
#Convert the dictionary to a geometry object
extentPoly_DD = geometry.Envelope(myExtent)

In [ ]:
#Project the extentPoly to the spatial reference of the imagery layer'
# recall that the "lc_sr" variable was defined above...
extentPoly_prj = geometry.project([extentPoly_DD],4326,lc_sr)[0]
extentPoly_prj

In [ ]:
#Have a look at our layer, subset to the extent we provided
lc_lyr.extent =  extentPoly_prj
lc_lyr

In [ ]:
from arcgis.raster.functions import clip
lc_clip = clip(raster=lc_lyr,geometry=extentPoly_prj)

In [ ]:
lc_clip

In [ ]:
import pandas as pd
attDF = pd.DataFrame.from_records(lc_clip.attribute_table()['features'])
attDF = attDF['attributes'].apply(pd.Series).set_index('OBJECTID')
attDF.head()

In [ ]:
#Now we need to compute number of rows and columns contained in this extent
xSize = int((extentPoly_prj.xmax - extentPoly_prj.xmin) / xPixelSize)
ySize = int((extentPoly_prj.ymax - extentPoly_prj.ymin) / xPixelSize)
xSize,ySize

* Use the imagery layer's [`export_image`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.export_image) function to extract a subset of data.

In [ ]:
#Extract data
img = lc_lyr.export_image(bbox=extentPoly_prj,
                          size=[xSize,ySize],
                          f='image')
type(img)

In [ ]:
#Show the image
Image(img)

In [ ]:
#Or we can save the output directly to a file
savedimg = lc_lyr.export_image(bbox=extentPoly_prj,
                               save_folder='.', 
                               save_file='DurhamNLCD.tif',
                               size=[xSize,ySize],
                               export_format='tiff',
                               compression_quality=100,
                               f='image'
                              )

In [ ]:
#Or we can examine properties of our saved image as a JSON object
img_json = lc_lyr.export_image(bbox=extentPoly_prj,
                               f='json'
                              )
img_json